# Java API

### Rapid API

In [1]:
import java.net.URI;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;

HttpRequest request = HttpRequest.newBuilder()
		.uri(URI.create("https://nfl-schedule.p.rapidapi.com/v1/schedules"))
		.header("X-RapidAPI-Key", "9fb1283360mshedc514375b603d6p156a26jsna7cd4ca5744a")
		.header("X-RapidAPI-Host", "nfl-schedule.p.rapidapi.com")
		.method("GET", HttpRequest.BodyPublishers.noBody())
		.build();
HttpResponse<String> response = HttpClient.newHttpClient().send(request, HttpResponse.BodyHandlers.ofString());
System.out.println(response.body());

{"message":"NFL Scheduled games","data":[{"date":"2022-10-03T00:20Z","venue":"Raymond James Stadium","name":"Kansas City Chiefs at Tampa Bay Buccaneers","shortName":"KC @ TB","awayTeam":{"name":"Chiefs","score":"7"},"homeTeam":{"name":"Buccaneers","score":"3"}},{"date":"2022-10-02T13:30Z","venue":"Tottenham Hotspur Stadium","name":"Minnesota Vikings at New Orleans Saints","shortName":"MIN @ NO","awayTeam":{"name":"Vikings","score":"28"},"homeTeam":{"name":"Saints","score":"25"}},{"date":"2022-10-02T17:00Z","venue":"Mercedes-Benz Stadium","name":"Cleveland Browns at Atlanta Falcons","shortName":"CLE @ ATL","awayTeam":{"name":"Browns","score":"20"},"homeTeam":{"name":"Falcons","score":"23"}},{"date":"2022-10-02T17:00Z","venue":"AT&T Stadium","name":"Washington Commanders at Dallas Cowboys","shortName":"WSH @ DAL","awayTeam":{"name":"Commanders","score":"10"},"homeTeam":{"name":"Cowboys","score":"25"}},{"date":"2022-10-02T17:00Z","venue":"Ford Field","name":"Seattle Seahawks at Detroit Li

### Web Service

In [5]:
import java.net.URI;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;
import java.util.Date;
import java.util.HashMap;

import org.json.simple.JSONObject;
import org.json.simple.parser.JSONParser;
import org.springframework.http.HttpStatus;
import org.springframework.http.ResponseEntity;
import org.springframework.web.bind.annotation.*;

@RestController // annotation to create a RESTful web services
@RequestMapping("/api/covid")  //prefix of API
public class CovidApiController {
    private JSONObject body; //last run result
    private HttpStatus status; //last run status
    String last_run = null; //last run day of month

    // GET Covid 19 Stats
    @GetMapping("/daily")   //added to end of prefix as endpoint
    public ResponseEntity<JSONObject> getCovid() {

        //calls API once a day, sets body and status properties
        String today = new Date().toString().substring(0,10); 
        if (last_run == null || !today.equals(last_run))
        {
            try {  //APIs can fail (ie Internet or Service down)

                //RapidAPI header
                HttpRequest request = HttpRequest.newBuilder()
                    .uri(URI.create("https://nfl-schedule.p.rapidapi.com/v1/schedules"))
                    .header("X-RapidAPI-Key", "9fb1283360mshedc514375b603d6p156a26jsna7cd4ca5744a")
                    .header("X-RapidAPI-Host", "nfl-schedule.p.rapidapi.com")
                    .method("GET", HttpRequest.BodyPublishers.noBody())
                    .build();

                //RapidAPI request and response
                HttpResponse<String> response = HttpClient.newHttpClient().send(request, HttpResponse.BodyHandlers.ofString());

                //JSONParser extracts text body and parses to JSONObject
                this.body = (JSONObject) new JSONParser().parse(response.body());
                this.status = HttpStatus.OK;  //200 success
                this.last_run = today;
            }
            catch (Exception e) {  //capture failure info
                HashMap<String, String> status = new HashMap<>();
                status.put("status", "RapidApi failure: " + e);

                //Setup object for error
                this.body = (JSONObject) status;
                this.status = HttpStatus.INTERNAL_SERVER_ERROR; //500 error
                this.last_run = null;
            }
        }

        //return JSONObject in RESTful style
        return new ResponseEntity<>(body, status);
    }
}